Build a dataframe of the postal code of each neighborhood along with borough name and neighborhood name in Toronto Get the Geographical coordinates of the neighborhoods in toronto Clustering the neighborhoods

In [14]:
!pip install folium
!pip install geopy

Part 1

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [15]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [16]:
wiki='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wiki= requests.get(wiki).text
soup = BeautifulSoup(raw_wiki,'xml')

In [17]:
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []
# print(table)
# extracting a clean form of the table
for tr_cell in table.find_all('tr'): 
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
        counter +=1
        if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'):
            continue          
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

In [18]:
unique_p = set(Postcode)
print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []
for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

num of unique Postal codes: 77


In [19]:
toronto_dict = {'Postal Code':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.to_csv('toronto_part1.csv')
df_toronto.head()

,Postal Code,Borough,Neighbourhood
0,M1R,Scarborough,"Maryvale, Wexford"
1,M6P,West Toronto,High Park
2,M4Y,Downtown Toronto,Church and Wellesley
3,M8Y,Etobicoke,"Humber Bay, Mimico NE, Old Mill South, The Que..."
4,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre"


In [20]:
df_toronto.shape

(77, 3)

Part 2

Using the Geocoder package or the csv file to create the dataframe.

In [21]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

In [22]:
TorontoPostalCodes = df_toronto.set_index("Postal Code")
TorontoPostalCodes.rename_axis("Postal Code", axis='index', inplace=True)
TorontoPostalCodes.head()

,Borough,Neighbourhood
Postal Code,,
M1R,Scarborough,"Maryvale, Wexford"
M6P,West Toronto,High Park
M4Y,Downtown Toronto,Church and Wellesley
M8Y,Etobicoke,"Humber Bay, Mimico NE, Old Mill South, The Que..."
M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre"


In [23]:
toronto_geocsv = 'https://cocl.us/Geospatial_data'
!wget -q -O 'toronto_m.geospatial_data.csv' toronto_geocsv
geocsv_data = pd.read_csv(toronto_geocsv).set_index("Postal Code")
geocsv_data.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [24]:
df = pd.merge(df_toronto, geocsv_data, how='inner', on = 'Postal Code')
#save the file incase you change the content by chance.
df.to_csv('toronto_df.csv')
df = pd.read_csv('toronto_df.csv')
df.head(12)

,Unnamed: 0,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,0,M1R,Scarborough,"Maryvale, Wexford",43.750071,-79.295849
1,1,M6P,West Toronto,High Park,43.661608,-79.464763
2,2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,3,M8Y,Etobicoke,"Humber Bay, Mimico NE, Old Mill South, The Que...",43.636258,-79.498509
4,4,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576
5,5,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
6,6,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049
7,7,M1G,Scarborough,Woburn,43.770992,-79.216917
8,8,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325
9,9,M9B,Etobicoke,"Islington, Princess Gardens, West Deane Park",43.650943,-79.554724
